<a href="https://colab.research.google.com/github/Matheus-Homem/future-sales-prediction/blob/main/notebook/Future_Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Imports

## 0.1. Libraries

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

In [13]:
import warnings
warnings.filterwarnings('ignore')

## 0.2. Loading Data

In [14]:
# Importing data from .csv
item_categories = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/item_categories%20(translated).csv")
items = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/items%20(translated).csv")
sales_train = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/sales_train.csv")
shops = pd.read_csv("https://github.com/Matheus-Homem/future-sales-prediction/raw/main/data/shops%20(translated).csv")

In [15]:
# First row of 'item_categories'
item_categories.loc[0]

item_category_name         PC - Гарнитуры/Наушники
item_category_name_en    PC - Headset / Headphones
item_category_id                                 0
Name: 0, dtype: object

In [16]:
# First row of 'items'
items.loc[0]

item_name                ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D
item_name_en        Lucky In the power of the puffy (reservoir.) D
item_id                                                          0
item_category_id                                                40
Name: 0, dtype: object

In [17]:
# First row of 'sales_train'
sales_train.loc[0]

date              02.01.2013
date_block_num             0
shop_id                   59
item_id                22154
item_price               999
item_cnt_day               1
Name: 0, dtype: object

In [18]:
# First row of 'shops'
shops.loc[0]

shop_name          !Якутск Орджоникидзе, 56 фран
shop_name_en    ! Yakutsk Ordzhonikidze, 56 fran
shop_id                                        0
Name: 0, dtype: object

## 0.3. Helper Functions

# 1. Data Description

## 1.1. Merging Tables

In [22]:
df1 = pd.merge(items,item_categories,on='item_category_id')
df1 = pd.merge(sales_train,df1,on='item_id')
df1 = pd.merge(df1,shops,on='shop_id')

cols_drop = ['item_name','item_category_name','shop_name']
df1 = df1.drop(cols_drop, axis=1)

df1.rename(columns={'item_name_en': 'item_name',
                   'item_category_name_en': 'item_category_name',
                   'shop_name_en': 'shop_name'},
                   inplace=True)

df1.sample().T

,982809
date,13.09.2014
date_block_num,20
shop_id,42
item_id,8662
item_price,149
item_cnt_day,1
item_name,Barboskina Issue 3: Real Guy (Region)
item_category_id,40
item_category_name,Cinema - DVD.
shop_name,"SPB TK ""Nevsky Center"""


## 1.2. Data Dimensions

In [23]:
print('Number of Rows: {:,}'.format(df1.shape[0]))
print('Number of Columns: ',df1.shape[1])

Number of Rows: 2,935,849
Number of Columns:  10


## 1.3. Data Types

In [24]:
df1.dtypes

date                   object
date_block_num          int64
shop_id                 int64
item_id                 int64
item_price            float64
item_cnt_day          float64
item_name              object
item_category_id        int64
item_category_name     object
shop_name              object
dtype: object

## 1.4. Change Data Types

In [25]:
df1['date'] = pd.to_datetime(df1['date'])

## 1.5. NA Checking

In [26]:
df1.isna().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
item_name             0
item_category_id      0
item_category_name    0
shop_name             0
dtype: int64

## 1.6. Fillout NA

- None of the columns have null values;

## 1.7. Descriptive Statistical

In [27]:
num_attributes = df1.select_dtypes(include=['int64','float64'])
cat_attributes = df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

### 1.7.1. Numerical Attributes

In [28]:
# Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# Contacatenate
n = pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
n.columns = ['Attributes','Min','Max','Range','Mean','Median','Std','Skew','Kurtosis']

In [29]:
n

,Attributes,Min,Max,Range,Mean,Median,Std,Skew,Kurtosis
0,date_block_num,0.0,33.0,33.0,14.569911,14.0,9.422986,0.203858,-1.082869
1,shop_id,0.0,59.0,59.0,33.001728,31.0,16.226970,-0.072361,-1.025358
2,item_id,0.0,22169.0,22169.0,10197.227057,9343.0,6324.296277,0.257174,-1.225210
3,item_price,-1.0,307980.0,307981.0,890.853233,399.0,1729.799336,10.750423,445.532826
4,item_cnt_day,-22.0,2169.0,2191.0,1.242641,1.0,2.618834,272.833162,177478.098774
5,item_category_id,0.0,83.0,83.0,40.001383,40.0,17.100756,0.318283,-0.525158


### 1.7.2. Categorical Attributes

In [30]:
# Unique values
cat_attributes.apply(lambda x: x.unique().shape[0])

item_name             21720
item_category_name       84
shop_name                60
dtype: int64

# 2. Feature Engineering

In [31]:
df2 = df1.copy()

## 2.1. Hypothesis Mind Map

## 2.2. List of Hypothesis

## 2.3. Feature Engineering

In [85]:
# Grouping item categories
unique_categories = list(df2['item_category_name'].unique())
unique_categories.sort()
simple_categories = list(map(lambda x: x.split('-')[0], unique_categories))
simple_categories = list(map(lambda x: x.split('(')[0].rstrip(), simple_categories))
category_dict = dict(zip(unique_categories, simple_categories))
df2['simple_categories'] = df2['item_category_name'].map(category_dict)

# Extracting city name from shop name
unique_shops = df2['shop_name'].unique()
unique_shops.sort()
cities = unique_shops
cities[:2] = list(map(lambda x: x[2:],cities[:2]))
cities = list(map(lambda x: x.split(' ')[0],cities))
cities[2] = cities[2][3:]
cities = list(map(lambda x: x.capitalize(),cities))
cities_dict = dict(zip(unique_shops, cities))
df2['city'] = df2['shop_name'].map(cities_dict)

# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.weekofyear

# year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

In [86]:
df2.sample(3).T

,238370,2686967,939536
date,2013-12-16 00:00:00,2013-04-08 00:00:00,2014-07-15 00:00:00
date_block_num,11,7,18
shop_id,25,45,54
item_id,17473,7856,15868
item_price,299,799,3399
item_cnt_day,1,2,1
item_name,Damned Path (BD),World of Warcraft. Game Time Payment Map (Onli...,"Set for poker ""Texas Holdem"" 300 chips"
item_category_id,37,28,64
item_category_name,Cinema - Blu-ray,PC Games - Additional Editions,Gifts - Board Games
shop_name,"Moscow TRK ""Atrium""","Samara shopping center ""Parkhouse""","Khimki shopping center ""Mega"""


# 3. Feature Filtering

In [87]:
df3 = df2.copy()

## 3.1. Rows Filtering

In [90]:
df3 = df3[df3['item_cnt_day'] > 0]

## 3.2. Columns Filtering

In [91]:
cols_drop = ['shop_id','item_id','item_category_id']
df3.drop(cols_drop,axis=1,inplace=True)
list(df3.columns)

['date',
 'date_block_num',
 'item_price',
 'item_cnt_day',
 'item_name',
 'item_category_name',
 'shop_name',
 'simple_categories',
 'city',
 'year',
 'month',
 'day',
 'week_of_year',
 'year_week']

In [95]:
df3['date'].sub(df3['date'].min()).dt.days // 7 + 1

0            5
1           20
2            4
3           35
4            8
          ... 
2935844    148
2935845    148
2935846    147
2935847    106
2935848    146
Name: date, Length: 2928493, dtype: int64

In [96]:
min(df3['date'])

Timestamp('2013-01-01 00:00:00')

In [97]:
max(df3['date'])

Timestamp('2015-12-10 00:00:00')